In [122]:
import numpy as np

from scipy import ndimage

import itertools

https://adventofcode.com/2020

# Day 17

In [5]:
input17 = """..##.#.#
.#####..
#.....##
##.##.#.
..#...#.
.#..##..
.#...#.#
#..##.##"""

In [51]:
def map_to_array(astmap):
    arr2d = np.array([[e=='#' for e in row] for row in astmap.split('\n')]).T
    return arr2d.copy()

def array_to_map(arr, marker='#'):
    buffer = []
    for row in arr.T:
        for element in row:
            buffer.append(marker if element else '.')
        buffer.append('\n')
    return ''.join(buffer)

## Part 1

In [53]:
test_input = """.#.
..#
###"""
testarr = map_to_array(test_input)
print(array_to_map(testarr))

.#.
..#
###



In [65]:
boot_field = np.zeros((testarr.shape[0] + 6*2, testarr.shape[1] + 6*2, 1 + 6*2), dtype=bool)
boot_field[6:9, 6:9, 6] = testarr
print(array_to_map(boot_field[..., 6]))

...............
...............
...............
...............
...............
...............
.......#.......
........#......
......###......
...............
...............
...............
...............
...............
...............



In [101]:
neighbor_kernel = [[[1,1,1],
                    [1,1,1],
                    [1,1,1]],
                   [[1,1,1],
                    [1,0,1],
                    [1,1,1]],
                   [[1,1,1],
                    [1,1,1],
                    [1,1,1]]]
neighbor_kernel = np.array(neighbor_kernel)
assert np.sum(neighbor_kernel) == 26

def do_step(initial_field, nsteps=1):
    if nsteps==0:
        return initial_field.reshape(initial_field.shape[0], initial_field.shape[1], 1)
    
    boot_field = np.zeros((initial_field.shape[0] + nsteps*2, 
                           initial_field.shape[1] + nsteps*2, 
                           1 + nsteps*2), dtype=int)
    boot_field[nsteps:(nsteps + initial_field.shape[0]), 
               nsteps:(nsteps + initial_field.shape[1]), 
               nsteps] = initial_field.astype(int)
    
    for i in range(nsteps):
        corr = ndimage.correlate(boot_field, neighbor_kernel, mode='constant')
        new_field = boot_field & (corr==2)
        new_field[corr==3] = True
        boot_field = new_field.astype(int)
    
    return boot_field

step3 = do_step(testarr, nsteps=3)
                     
                     
assert array_to_map(step3[1:-1, 2:, 3]) == """...#...
.......
#......
.......
.....##
.##.#..
...#...
"""

assert np.sum(do_step(testarr, nsteps=6)) == 112

In [102]:
np.sum(do_step(map_to_array(input17), nsteps=6))

213

## Part 2 

In [105]:
neighbor_kernel4 = np.ones((3, 3, 3, 3), dtype=int)
neighbor_kernel4[1,1,1,1] = 0
neighbor_kernel4 = np.array(neighbor_kernel4)
assert np.sum(neighbor_kernel4) == 80

In [117]:
def do_step4(initial_field, nsteps=1):
    if nsteps==0:
        return initial_field.reshape(initial_field.shape[0], initial_field.shape[1], 1, 1)
    
    boot_field = np.zeros((initial_field.shape[0] + nsteps*2, 
                           initial_field.shape[1] + nsteps*2, 
                           1 + nsteps*2, 1 + nsteps*2), dtype=int)
    boot_field[nsteps:(nsteps + initial_field.shape[0]), 
               nsteps:(nsteps + initial_field.shape[1]), 
               nsteps, nsteps] = initial_field.astype(int)
    
    for i in range(nsteps):
        corr = ndimage.correlate(boot_field, neighbor_kernel4, mode='constant')
        new_field = boot_field & (corr==2)
        new_field[corr==3] = True
        boot_field = new_field.astype(int)
    
    return boot_field

assert array_to_map(do_step4(testarr, 2)[1:-1, 2:, 2, 0]) == """###..
##.##
#...#
.#..#
.###.
""""""

In [131]:
assert np.sum(do_step4(testarr, nsteps=6)) == 848

In [132]:
np.sum(do_step4(map_to_array(input17), nsteps=6))

1624

# Day 18

In [3]:
with open('input18') as f:
    input18 = f.read().strip()

## Part 1

For loop over lines is probably ok for modest-size inputs

In [101]:
def find_subexpressions(s):
    sub_exprs = {}
    if '(' in s:
        paren_level = 0
        for i, si in enumerate(s):
            if si=='(':
                paren_level += 1
                if paren_level == 1:
                    sub_exprs['current'] = i
            elif si == ')':
                paren_level -= 1
                if paren_level == 0:
                    sub_exprs[sub_exprs['current']] = i
                    del sub_exprs['current']
        if paren_level != 0:
            raise ValueError('parens not matched!')
    return sub_exprs
    

def evaluate_expression(s):
    sub_exprs = find_subexpressions(s)
    # accumulate
    result = 0
    
    operator = 'start'
    nums = tuple('1234567890')
    i = 0
    while i < len(s):
        si = s[i]
        
        if i in sub_exprs:
            substr = s[i+1:sub_exprs[i]]
            subres = evaluate_expression(substr)
            if operator == 'start':
                result = subres
            elif operator is None:
                raise ValueError('syntax error - num does not follow operator')
            elif operator == '+':
                result += subres
            elif operator == '*':
                result *= subres
            operator = None
            j = sub_exprs[i]+2
            i = j
            continue
        elif si in ('*', '+'):
            operator = si
        elif si in nums:
            if operator == 'start':
                result += int(si)
            elif operator is None:
                raise ValueError('syntax error - num does not follow operator')
            elif operator == '+':
                result += int(si)
            elif operator == '*':
                result *= int(si)
                operator = None
        elif si == ' ':
            pass
        else:
            raise ValueError(f"don't know what to do with character {si}")
        i += 1
    return result

assert evaluate_expression('9 + 3 * 4') == (9 + 3) * 4
assert evaluate_expression('9 + (3 * 4)') == 9 + 3 * 4 
assert evaluate_expression('1 + (3 + 2) + ((1 + 2) + 3)') == 1 + (3 + 2) + ((1 + 2) + 3)

In [102]:
%timeit evaluate_expression('1 + (3 + 2) + ((1 + 2) + 3)')

9.38 µs ± 114 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [103]:
sum([evaluate_expression(line) for line in input18.split('\n')])

7147789965219

## Part 1b

As feared, the above doesn't really work with part 2...

In [148]:
NUMS = tuple('1234567890')
OPERATORS = ('*', '+')

class Number:
    def __init__(self, num):
        self.num = int(num)
    def __call__(self):
        return self.num
    
class Expression:
    def __init__(self, s):
        self.expressions = []
        self.operators = []
        
        sub_exprs = find_subexpressions(s)
        
        i = 0
        while i < len(s):
            si = s[i]
            if si in NUMS:
                self.expressions.append(Number(si))
            if si in OPERATORS:
                self.operators.append(si)
            elif i in sub_exprs:
                self.expressions.append(self.__class__(s[i+1:sub_exprs[i]]))
                j = sub_exprs[i]+2
                i = j
                continue
            i += 1
        if len(self.expressions)-1!= len(self.operators):
            raise ValueError(f'operators and expressions not matched! expressions:{self.expressions}, operators:{self.operators}')
    def __call__(self):
        evaled = [e() for e in self.expressions]
        result = evaled[0]
        for i, op in enumerate(self.operators):
            if op == '+':
                result += evaled[i+1]
            elif op == '*':
                result *= evaled[i+1]
            else:
                raise ValueError(f'Invalid operator "{op}"')
        return result
            
assert Expression('2 * 3 + (4 * 5)')() == 26
assert Expression('5 + (8 * 3 + 9 + 3 * 4 * 3)')() == 437
assert Expression('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))')() == 12240
assert Expression('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2')() == 13632

In [150]:
assert sum([Expression(line)() for line in input18.split('\n')]) == 7147789965219

## Part 2 

In [169]:
class PlusPrecedenceExpression(Expression):
    def __call__(self):
        evaled = [e() for e in self.expressions]
        ops = self.operators.copy()
        result = evaled[0]
        for i in range(len(self.operators))[::-1]:
            op = ops[i]
            if op == '+':
                result = evaled[i] + evaled[i+1]
                del evaled[i+1]
                evaled[i] = result
                del ops[i]
            elif op == '*':
                pass
            else:
                raise ValueError(f'Invalid operator "{op}"')
        return np.prod(evaled)
    
assert PlusPrecedenceExpression('1 + (2 * 3) + (4 * (5 + 6))')() == 51
assert PlusPrecedenceExpression('2 * 3 + (4 * 5)')() == 46
assert PlusPrecedenceExpression('5 + (8 * 3 + 9 + 3 * 4 * 3)')() == 1445
assert PlusPrecedenceExpression('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))')() == 669060
assert PlusPrecedenceExpression('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2')() == 23340

In [170]:
sum([PlusPrecedenceExpression(line)() for line in input18.split('\n')])

136824720421264

# Day 19

In [2]:
with open('input19') as f:
    input19 = f.read().strip()

## Part 1

In [55]:
basic_test_input = '''0: 1 2
1: "a"
2: 1 3 | 3 1
3: "b"'''

def parse_input(s):
    rules = {}
    messages = []
    in_rules = True
    for l in s.strip().split('\n'):
        if l.strip() == '':
            in_rules = False
        elif in_rules:
            code, rule = l.split(':')
            if '"' in rule:
                rules[int(code)] = rule.replace('"', '').strip()
            else:
                rules[int(code)] = [tuple([int(n) for n in grp.split()]) for grp in rule.split('|')]
        else:
            messages.append(l.strip())
    return rules, messages
            
basic_rules, messages = parse_input(basic_test_input)  
basic_rules, messages

({0: [(1, 2)], 1: 'a', 2: [(1, 3), (3, 1)], 3: 'b'}, [])

In [94]:
def check_message(message, rules, start_index):
    my_rules = rules[start_index]
    isstring = isinstance(my_rules, str)
        
    if isinstance(my_rules, str):
        if message[0] == my_rules:
            return message[1:]
        else:
            return False
    else:
        for ruleset_or in my_rules:
            current_message = message
            for rule in ruleset_or:
                if current_message == '' or current_message is True:
                    return False
                current_message = check_message(current_message, rules, rule)
                if current_message is False:
                    break
            else:
                if current_message == '':
                    return True
                else:
                    return current_message
        else:
            return False
        
check_message('aab', basic_rules, 0), check_message('aba', basic_rules, 0), check_message('abb', basic_rules, 0)

(True, True, False)

In [95]:
test_input = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb"""
rules, messages = parse_input(test_input)
rules, messages

({0: [(4, 1, 5)],
  1: [(2, 3), (3, 2)],
  2: [(4, 4), (5, 5)],
  3: [(4, 5), (5, 4)],
  4: 'a',
  5: 'b'},
 ['ababbb', 'bababa', 'abbbab', 'aaabbb', 'aaaabbb'])

In [96]:
assert np.all(np.array([check_message(message, rules, 0) is True for message in messages]) == 
              [True, False, True, False, False])

In [97]:
rules, messages = parse_input(input19)
sum([check_message(message, rules, 0) is True for message in messages])

156

## Part 2 

In [98]:
updated_input19 = input19.replace('8: 42', '8: 42 | 42 8').replace('11: 42 31', '11: 42 31 | 42 11 31')

In [99]:
rules, messages = parse_input(updated_input19)
sum([check_message(message, rules, 0) is True for message in messages])

199

Hmm, it's not doing infinite recursion.  Clearly an algorithm flaw

# Day 20

In [273]:
with open('input20') as f:
    input20 = f.read().strip()

## Part 1

In [274]:
test_input = """Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###..."""

def parse_input(s):
    tiles = {}
    tilenum = None
    tiles = {}
    for line in s.strip().split('\n'):
        if line.strip() == '':
            pass
        elif line.startswith('Tile'):
            tilenum = int(line.split(':')[0].split('Tile ')[1])
            tiles[tilenum] = []
        else:
            tiles[tilenum].append([c=='#' for c in line])
    for nm in tiles:
        tiles[nm] = np.array(tiles[nm])
    return tiles

test_tiles = parse_input(test_input)
test_tiles.keys()

dict_keys([2311, 1951, 1171, 1427, 1489, 2473, 2971, 2729, 3079])

In [275]:
def tile_to_map(tile):
    chars = []
    for row in tile:
        for e in row:
            chars.append('#' if e else '.')
        chars.append('\n')
    return ''.join(chars)
print(tile_to_map(test_tiles[2311]))

..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###



In [276]:
def make_edges_arrays(tiles):
    #first for each tile make an array of all the different edge combinations:
    # [left, top , right, bottom] + [l_flipped, t_flipped, r_flipped, bottom_flipped]
    edge_arrays = {}
    for nm, tile in tiles.items():
        edges = [tile[0, :], tile[:, 0], tile[-1, :], tile[:, -1]]
        edge_arrays[nm] = np.array(edges + [e[::-1] for e in edges])
    return edge_arrays

edge_arrays = make_edges_arrays(test_tiles)
a1 = edge_arrays[2311]
a2 = edge_arrays[1951]

In [277]:
def find_matches(tiles):
    edge_arrays = make_edges_arrays(tiles)
    
    matches = {}
    for tile, edges in edge_arrays.items():
        for i, edge in enumerate(edges[:4]):  # only match the "forward" versions since otherwise there's a double flip
            for tile2, edges2 in edge_arrays.items():
                if tile2 == tile:
                    continue
                for j, edge2 in enumerate(edges2):
                    if np.all(edge == edge2):
                        matches.setdefault(tile, []).append((i, tile2, j))
    return matches

%time find_matches(test_tiles)

CPU times: user 11.5 ms, sys: 11 µs, total: 11.5 ms
Wall time: 9.71 ms


{2311: [(0, 1427, 2), (1, 1951, 3), (3, 3079, 5)],
 1951: [(0, 2729, 2), (3, 2311, 1)],
 1171: [(0, 2473, 1), (3, 1489, 7)],
 1427: [(0, 1489, 2), (1, 2729, 3), (2, 2311, 0), (3, 2473, 2)],
 1489: [(1, 2971, 3), (2, 1427, 0), (3, 1171, 7)],
 2473: [(1, 1171, 0), (2, 1427, 3), (3, 3079, 6)],
 2971: [(2, 2729, 0), (3, 1489, 1)],
 2729: [(0, 2971, 2), (2, 1951, 0), (3, 1427, 1)],
 3079: [(1, 2311, 7), (2, 2473, 7)]}

In [278]:
len(make_edges_arrays(test_tiles)), len(make_edges_arrays(parse_input(input20)))

(9, 144)

factor of ~10 larger is only 100 slower, so not too slow.  Continue with method to make matches

Cheat! The corners are the ones that have exactly two matches...

In [134]:
test_matches = find_matches(test_tiles)
corners = [tile for tile in test_matches if len(test_matches[tile]) == 2]
assert len(corners) == 4
assert np.prod(corners) == 20899048083289

In [135]:
test_matches

{2311: [(0, 1427, 2), (1, 1951, 3), (3, 3079, 5)],
 1951: [(0, 2729, 2), (3, 2311, 1)],
 1171: [(0, 2473, 1), (3, 1489, 7)],
 1427: [(0, 1489, 2), (1, 2729, 3), (2, 2311, 0), (3, 2473, 2)],
 1489: [(1, 2971, 3), (2, 1427, 0), (3, 1171, 7)],
 2473: [(1, 1171, 0), (2, 1427, 3), (3, 3079, 6)],
 2971: [(2, 2729, 0), (3, 1489, 1)],
 2729: [(0, 2971, 2), (2, 1951, 0), (3, 1427, 1)],
 3079: [(1, 2311, 7), (2, 2473, 7)]}

In [281]:
matches = find_matches(parse_input(input20))

In [282]:
corners = [tile for tile in matches if len(matches[tile]) == 2]
assert len(corners) == 4
np.prod(corners)

15405893262491

## Part 2 

Arg the cheat isn't good enough - need to construct the whole picture...

In [138]:
len(matches)**0.5

12.0

12 x 12 means there should be 4 corners (2 matches), 10x4 = 40 edges(3 matches), and the remainder (100) should have 4 matches:

In [139]:
nmatches = [len(v) for v in matches.values()]
nmatches.count(2), nmatches.count(3), nmatches.count(4)

(4, 40, 100)

Hooray! A unique solution. Just need to construct it. Can declare any corner to be the "first" corner.  Just pick the 0th one:

In [270]:
def arrange_tiles(matches):
    sqsize = len(matches)**0.5
    assert int(sqsize) == sqsize
    sqsize = int(sqsize)
    
    corners = [tile for tile in matches if len(matches[tile]) == 2]
    tile_positions = (-sqsize-1)*np.ones((sqsize, sqsize), dtype=int)
    tile_positions[0, 0] = corners[0]
    
    next_edge_index = None
    for i in range(len(tile_positions)-1):
        tile = tile_positions[i, 0]
        for ei1, nexttile, ei2 in matches[tile]:
            if nexttile not in tile_positions.ravel() and (next_edge_index is None or ei1%2 == next_edge_index):
                tile_positions[i+1, 0] = nexttile
                next_edge_index = ei2 % 2
                break
        else:
            assert False
    # now do the same but along all y-axes
    for i in range(tile_positions.shape[0]):
        next_edge_index = None
        for j in range(tile_positions.shape[1]-1):
            tile = tile_positions[i, j]
            for ei1, nexttile, ei2 in matches[tile]:
                if nexttile not in tile_positions.ravel() and (next_edge_index is None or ei1%2 == next_edge_index):
                    tile_positions[i, j+1] = nexttile
                    next_edge_index = ei2 % 2
                    break
            else:
                assert False
                
    # check the corners make sense
    for i, j in [(0,0), (0, -1), (-1, 0), (-1, -1)]:
        assert tile_positions[i, j] in corners, 'Corners not consistent!'

    return tile_positions

test_matches = find_matches(test_tiles)
test_arrangement = arrange_tiles(test_matches)

assert np.all(test_arrangement == np.array(
      [[1951, 2311, 3079],
       [2729, 1427, 2473],
       [2971, 1489, 1171]]))

Now need to figure out how they're oriented to align

In [272]:
def reorient_tile(tile, rots, flip):
    for _ in range(rots):
        tile = np.rot90(tile)
        
    if flip == 1:
        return tile[::-1]
    elif flip == 2:
        return tile[:, ::-1]
    elif flip == 0:
        return tile
    else:
        raise ValueError(f'Invalid flip value {flip}')

transformation_edge_map = {(0,0):{0:0, 1:1, 2:2, 3:3},
                   (1,0):{0:1, 1:2, 2:3, 3:0},
                   (2,0):{0:2, 1:3, 2:0, 3:1},
                   (3,0):{0:3, 1:0, 2:1, 3:2},
                   (0,1):{0:2, 1:1, 2:0, 3:3},
                   (1,1):{0:1, 1:0, 2:3, 3:2},
                   (2,1):{0:0, 1:3, 2:2, 3:1},
                   (3,1):{0:3, 1:2, 2:1, 3:0}
                   }
def find_orientation(tile_positions, matches):
    tile_orientations = {}
    for i in range(tile_positions.shape[0]):
        for j in range(tile_positions.shape[1]):
            tilenum = tile_positions[i, j]
            
            tile0 = tile1 = tile2 = tile3 = None
            if i>0:
                tile0 = tile_positions[i-1, j]
            if (i+1) < tile_positions.shape[0]:
                tile2 = tile_positions[i+1, j]
            if j>0:
                tile1 = tile_positions[i, j-1]
            if (j+1) < tile_positions.shape[1]:
                tile3 = tile_positions[i, j+1]
                
            matched_edges = {tilenum:edge_idx for edge_idx, tilenum, _ in matches[tilenum]}
            goal = np.array([i for i, t in zip(range(4),[tile0, tile1, tile2, tile3]) if t is not None])
            starting_edges = [matched_edges[t] for t in [tile0, tile1, tile2, tile3] if t is not None]
            for reorient, transform_map in transformation_edge_map.items():
                edges = np.array([transform_map[e] for e in starting_edges])
                if np.all(edges == goal):
                    tile_orientations[tilenum] = reorient
                    break
            else:
                raise ValueError(f'unsolvable tile! {tilenum}')
    return tile_orientations

def print_tile_array(tiles, tile_positions, tile_orientations):
    for i in range(tile_positions.shape[0]):
        multilines = {}
        for j in range(tile_positions.shape[1]):
            tilenum = tile_positions[i, j]
            retile = reorient_tile(tiles[tilenum], *tile_orientations[tilenum])
            tile_lines = tile_to_map(retile).split('\n')
            for k, line in enumerate(tile_lines):
                multilines.setdefault(k, []).append(line)
        for k in sorted(multilines):
            print(' '.join(multilines[k]))
        print('')
      

test_orientations = find_orientation(test_arrangement, test_matches)
print_tile_array(test_tiles, test_arrangement, test_orientations)

#...##.#.. ..###..### #.#.#####.
..#.#..#.# ###...#.#. .#..######
.###....#. ..#....#.. ..#.......
###.##.##. .#.#.#..## ######....
.###.##### ##...#.### ####.#..#.
.##.#....# ##.##.###. .#...#.##.
#...###### ####.#...# #.#####.##
.....#..## #...##..#. ..#.###...
#.####...# ##..#..... ..#.......
#.##...##. ..##.#..#. ..#.###...
  

#.##...##. ..##.#..#. ..#.###...
##..#.##.. ..#..###.# ##.##....#
##.####... .#.####.#. ..#.###..#
####.#.#.. ...#.##### ###.#..###
.#.####... ...##..##. .######.##
.##..##.#. ....#...## #.#.#.#...
....#..#.# #.#.#.##.# #.###.###.
..#.#..... .#.##.#..# #.###.##..
####.#.... .#..#.##.. .######...
...#.#.#.# ###.##.#.. .##...####
  

...#.#.#.# ###.##.#.. .##...####
..#.#.###. ..##.##.## #..#.##..#
..####.### ##.#...##. .#.#..#.##
#..#.#..#. ...#.#.#.. .####.###.
.#..####.# #..#.#.#.# ####.###..
.#####..## #####...#. .##....##.
##.##..#.. ..#...#... .####...#.
#.#.###... .##..##... .####.##.#
#...###... ..##...#.. ...#..####
..#.#....# ##.#.#.... ...##.....
  

In [286]:
tiles = parse_input(input20)
tile_matches = find_matches(tiles)
tile_arrangement = arrange_tiles(tile_matches)
tile_orientations = find_orientation(tile_arrangement, tile_matches)
print_tile_array(tiles, tile_arrangement, tile_orientations)

...###.#.. ..#####... ...#..##.. .#.#..#..# #..###.### #.###.###. .###.##### #.###...## ####.#...# ##..#..... ..###...## ##...#.##.
#.....#... ....#.#... ...#...... ..#....#.# #...#...## #..###.... ...#.#..#. ..###..#.# #....#...# ##....#..# #.....#.#. .......##.
........#. ......#..# ##.###..## ###..#.... .....###.. .#.......# #.#...#..# #..#....#. .......... .......... .#..##.### #..#.....#
.....#...# #..#..#.## #.#.#.#### #.#..#.... ..##.#...# #..#....## #...#.##.. .......... .......... ....##.##. ..##..#... .##.......
#......#.. .......... ...#.#...# #.###....# #.......#. .###..#... .......... .....##... ....#....# ##......#. .........# #.#......#
##......#. ...#...##. .....#.... ..##..#... ....#...## ##......#. ....#.#... .....#..#. .####..... ......#... .......... ...#..#...
#...#....# ###...#..# ##...#...# ##.##..#.# #.##.#.##. .#.###..#. ...#...#.# #.......#. .##......# #..#.#...# ###...#... ..#..#.#..
#........# #..#..#..# #...#...## #...##.... .......##. ..#.##...# #..##.#..#

In [321]:
def stitch_tiles(tiles, positions, orientations, border=None):
    def get_tile(tnm):
        if border:
            tile = tiles[tnm][border:-border, border:-border]
        else:
            tile = tiles[tnm]
        return reorient_tile(tile, *orientations[tnm])
    
    tile_lsts = [[get_tile(positions[i, j]) for j in range(positions.shape[0])] for i in range(positions.shape[1])]
    return np.block(tile_lsts)
print(tile_to_map(stitch_tiles(test_tiles, test_arrangement, test_orientations)))
test_stiched = stitch_tiles(test_tiles, test_arrangement, test_orientations, border=1)
print(tile_to_map(test_stiched))

#...##.#....###..####.#.#####.
..#.#..#.####...#.#..#..######
.###....#...#....#....#.......
###.##.##..#.#.#..########....
.###.#######...#.#######.#..#.
.##.#....###.##.###..#...#.##.
#...##########.#...##.#####.##
.....#..###...##..#...#.###...
#.####...###..#.......#.......
#.##...##...##.#..#...#.###...
#.##...##...##.#..#...#.###...
##..#.##....#..###.###.##....#
##.####....#.####.#...#.###..#
####.#.#.....#.########.#..###
.#.####......##..##..######.##
.##..##.#.....#...###.#.#.#...
....#..#.##.#.#.##.##.###.###.
..#.#......#.##.#..##.###.##..
####.#.....#..#.##...######...
...#.#.#.####.##.#...##...####
...#.#.#.####.##.#...##...####
..#.#.###...##.##.###..#.##..#
..####.#####.#...##..#.#..#.##
#..#.#..#....#.#.#...####.###.
.#..####.##..#.#.#.#####.###..
.#####..#######...#..##....##.
##.##..#....#...#....####...#.
#.#.###....##..##....####.##.#
#...###.....##...#.....#..####
..#.#....###.#.#.......##.....

.#.#..#.##...#.##..#####
###....#.#....#..#......
##.##.###.#.#..####

In [322]:
stiched_image = stitch_tiles(tiles, tile_arrangement, tile_orientations, border=1)
print(tile_to_map(stiched_image))
stiched_image.shape

.....#.....#.#....#......#....#....#...#..###.....#.#..#.###..#.....#...#....#.......#.#......##
.......#.....#..#.###..###..#.......###.#........#...#....#....#................#..##.##..#.....
....#.....#..#.#.#.#.###.#..#....##.#.....#....#...#.##....................##.##.##..#..##......
......#...........#.#....###...........####..#..............##.....#....#......#.........#......
#......#..#...##....#....##..#.....#...##......#...#.#......#..#####.........#............#..#..
...#....##...#..#...#...#.##..#..##.#.###.###..#..#...#........###........#.#...##...#...#..#.#.
..........#..#.....#...#...##.........##.#.##.....##.#...##....###...##....#..#......#.....##.#.
##...#.#...##......#.###.....#.#........##.#....#..#.#.......#..##.####.#.....##..........###..#
..#.#.....#..##......#.#.#...###.......#.....##...###.....#.............#...#........##..#.###..
............#....#....#.#.....#.##.##..#.#..........#.#.#..........#...##..##.#.#......#.......#
#.....##.......#........#....#

(96, 96)

In [320]:
monster_template = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """
monster_arr = np.array([[c=='#' for c in row] for row in monster_template.split('\n')])
monster_arr.shape

(3, 20)

In [333]:
print(tile_to_map(reorient_tile(test_stiched, 1, 1)))

.####...#####..#...###..
#####..#..#.#.####..#.#.
.#.#...#.###...#.##.##..
#.#.##.###.#.##.##.#####
..##.###.####..#.####.##
...#.#..##.##...#..#..##
#.##.#..#.#..#..##.#.#..
.###.##.....#...###.#...
#.####.#.#....##.#..#.#.
##...#..#....#..#...####
..#.##...###..#.#####..#
....#.##.#.#####....#...
..##.##.###.....#.##..#.
#...#...###..####....##.
.#.##...#.##.#.#.###...#
#.###.#..####...##..#...
#.###...#.##...#.######.
.###.###.#######..#####.
..##.#..#..#.#######.###
#.#..##.########..#..##.
#.#####..#.#...##..#....
#....##..#.#########..##
#...#.....#..##...###.##
#..###....##.#...##.##.#



That's the correct orientation to find 2 monsters, so lets check that we do:

In [337]:
assert np.sum(ndimage.correlate(test_stiched.astype(int), monster_arr, mode='constant').ravel() == np.sum(monster_arr)) == 0
assert np.sum(ndimage.correlate(reorient_tile(test_stiched, 1, 1).astype(int), monster_arr, mode='constant').ravel() == np.sum(monster_arr)) == 2

Infer the non-monster pixels from the counts remaining

In [341]:
assert (np.sum(test_stiched) - np.sum(monster_arr)*2) == 273

In [350]:
good_orients = []
for reorients in transformation_edge_map:
    tile = reorient_tile(full_image, *reorients).astype(int)
    monsters = ndimage.correlate(tile, monster_arr, mode='constant') == np.sum(monster_arr)
    nmonsters = np.sum(monsters)
    if nmonsters > 0:
        good_orients.append((nmonsters, reorients, tile))
    print('n_monsters in', reorients, nmonsters)
assert len(good_orients) == 1

n_monsters in (0, 0) 0
n_monsters in (1, 0) 0
n_monsters in (2, 0) 0
n_monsters in (3, 0) 33
n_monsters in (0, 1) 0
n_monsters in (1, 1) 0
n_monsters in (2, 1) 0
n_monsters in (3, 1) 0


In [351]:
np.sum(full_image) - np.sum(monster_arr)*good_orients[0][0]

2133

# Day 21

In [6]:
with open('input21') as f:
    input21 = f.read().strip()

## Part 1

In [116]:
test_input = """mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)"""
def parse_input(s):
    inall_pairs = []
    for line in s.strip().split('\n'):
        ingredients, allergens = line.split(' (contains ')
        ingredients = ingredients.split()
        allergens = allergens.replace(')', '').split(', ')
        inall_pairs.append((ingredients, allergens))
    return inall_pairs
test_inalls = parse_input(test_input)
test_inalls

[(['mxmxvkd', 'kfcds', 'sqjhc', 'nhms'], ['dairy', 'fish']),
 (['trh', 'fvjkl', 'sbzzf', 'mxmxvkd'], ['dairy']),
 (['sqjhc', 'fvjkl'], ['soy']),
 (['sqjhc', 'mxmxvkd', 'sbzzf'], ['fish'])]

In [77]:
def get_non_allergens(inalls):
    allergen_to_ingredient_map = {}
    for ings, aller in inalls:
        for a in aller:
            allergen_to_ingredient_map.setdefault(a, []).append(ings)

    all_ingredients = set()
    possible_allergen_sets = set()
    for allergen, inglists in allergen_to_ingredient_map.items():
        s1 = set(inglists[0])
        possible_allergen_sets = possible_allergen_sets.union(s1.intersection(*inglists))
        all_ingredients = all_ingredients.union(*inglists)
    return all_ingredients.difference(possible_allergen_sets)

non_allergens = get_non_allergens(test_inalls)

n = 0
for nall in non_allergens:
    for ings, _ in test_inalls:
        if nall in ings:
            n+= 1
assert n == 5

In [79]:
inalls = parse_input(input21)
non_allergens = get_non_allergens(inalls)

n = 0
for nall in non_allergens:
    for ings, _ in inalls:
        if nall in ings:
            n+= 1
n

2724

## Part 2 

In [80]:
def only_possible_allergens(inalls):
    non_allergens = get_non_allergens(inalls)
    
    only_allergens = []
    for ings, aller in inalls:
        only_allergens.append(([ing for ing in ings if ing not in non_allergens], aller))
    return only_allergens
only_possible_allergens(test_inalls)

[(['mxmxvkd', 'sqjhc'], ['dairy', 'fish']),
 (['fvjkl', 'mxmxvkd'], ['dairy']),
 (['sqjhc', 'fvjkl'], ['soy']),
 (['sqjhc', 'mxmxvkd'], ['fish'])]

In [106]:
def allergen_to_ingredients(inalls):
    sub_inalls = only_possible_allergens(inalls)
    allergen_to_ingredient_map = {}
    all_allergens = set()
    for ings, aller in sub_inalls:
        for a in aller:
            allergen_to_ingredient_map.setdefault(a, []).append(ings)
        all_allergens = all_allergens.union(aller)
            
    allergen_to_ingredient = {}
    while len(allergen_to_ingredient) < len(all_allergens):
        for a, ings in allergen_to_ingredient_map.items():
            overlap_set = set(ings[0]).intersection(*ings)
            if len(overlap_set) == 1:
                allergen_to_ingredient[a] = ing = overlap_set.pop()
                for a, inglist in allergen_to_ingredient_map.items():
                    for ings in inglist:
                        if ing in ings:
                            ings.remove(ing)
                break
        else:
            raise ValueError('Unsolvable!')
    return allergen_to_ingredient
            
allergen_to_ingredients(test_inalls)

{'dairy': 'mxmxvkd', 'fish': 'sqjhc', 'soy': 'fvjkl'}

In [108]:
a2i = allergen_to_ingredients(test_inalls)
assert ','.join([a2i[a] for a in sorted(a2i)]) == 'mxmxvkd,sqjhc,fvjkl'

In [118]:
a2i = allergen_to_ingredients(parse_input(input21))
','.join([a2i[a] for a in sorted(a2i)]) 

'xlxknk,cskbmx,cjdmk,bmhn,jrmr,tzxcmr,fmgxh,fxzh'

# Day 22

In [3]:
with open('input22') as f:
    input22 = f.read().strip()

## Part 1

In [8]:
test_input="""Player 1:
9
2
6
3
1

Player 2:
5
8
4
7
10"""

def parse_input(s):
    player_decks = {}
    player = None
    for l in s.split('\n'):
        if l.strip() == '':
            pass
        elif l.startswith('Player'):
            player = int(l.replace('Player ', '').replace(':', '').strip())
        elif player is None:
            raise ValueError('invalid start of decks')
        else:
            player_decks.setdefault(player, []).append(int(l.strip()))
    return player_decks
parse_input(test_input)

{1: [9, 2, 6, 3, 1], 2: [5, 8, 4, 7, 10]}

In [16]:
def play_round(decks):
    card1 = decks[1].pop(0)
    card2 = decks[2].pop(0)
    if card1 > card2:
        decks[1].append(card1)
        decks[1].append(card2)
        return 1
    elif card2 > card1:
        decks[2].append(card2)
        decks[2].append(card1)
        return 2
    else:
        raise NotImplementedError('ties are currently ambiguous')
        
test_decks = parse_input(test_input)
while all([len(deck) > 0 for deck in test_decks.values()]):
    winner = play_round(test_decks)


winning_hand = np.array(test_decks[winner])
assert np.sum(winning_hand*(np.arange(len(winning_hand))[::-1]+1)) == 306

In [50]:
decks = parse_input(input22)
while all([len(deck) > 0 for deck in decks.values()]):
    winner = play_round(decks)


winning_hand = np.array(decks[winner])
winner, np.sum(winning_hand*(np.arange(len(winning_hand))[::-1]+1))

(1, 32162)

## Part 2 

In [31]:
recursive_test = """
Player 1:
43
19

Player 2:
2
29
14"""

In [61]:
def encode_game(decks):
    s1 = ','.join([str(card) for card in decks[1]])
    s2 = ','.join([str(card) for card in decks[2]])
    return s1 + '|' + s2

def recursive_game(decks):
    past_games_encoded = set()
    while all([len(deck) > 0 for deck in decks.values()]):
        enc = encode_game(decks)
        if enc in past_games_encoded:
            return -1, decks
        else:
            past_games_encoded.add(enc)
            
        if decks[1][0] < len(decks[1]) and decks[2][0] < len(decks[2]):
            # recurse
            card1 = decks[1].pop(0)
            card2 = decks[2].pop(0)
            winner, recursive_hand = recursive_game({1: decks[1][:card1], 2: decks[2][:card2]})
            if winner == 1:
                decks[1].append(card1)
                decks[1].append(card2)
            elif winner == 2:
                decks[2].append(card2)
                decks[2].append(card1)
            elif winner == -1:
                # winner is 1 for recursion purposes?
                decks[1].append(card1)
                decks[1].append(card2)
            else:
                raise ValueError(f'invalid winner {winner}')

        else:
            winner = play_round(decks)
            
    
    return winner, decks[winner]

assert recursive_game(parse_input(recursive_test))[0] == -1
winner, hand = recursive_game(parse_input(test_input))
assert winner == 2
assert np.sum(hand*(np.arange(len(hand))[::-1]+1)) == 291

In [62]:
winner, hand = recursive_game(parse_input(input22))
if winner == -1:
    hand = hand[1]
winner, np.sum(hand*(np.arange(len(hand))[::-1]+1))

(1, 32534)

# Day 23

In [7]:
with open('input23') as f:
    input23 = [int(c) for c in f.read().strip()]

## Part 1

In [372]:
test_input = [int(c) for c in '389125467']

In [88]:
def do_moves(icups, moves=100, verbose=False, progress=None):
    cups = icups.copy()
    
    mincups = min(cups)
    maxcups = max(cups)
    
    if progress is None:
        progress = lambda x:x
    
    idx = 0
    for i in progress(range(moves)):
        current_cup = cups[idx]
        
        picked_up = []
        for j in range(3):
            picked_up.append(cups[(idx+j+1) % len(cups)])
            
        dest = current_cup - 1
        if dest < mincups:
            dest = maxcups
        while dest in picked_up:
            dest -= 1
            if dest < mincups:
                dest = maxcups
        if verbose:
            print(cups, idx, dest, picked_up)
                
        for pcup in picked_up:
            cups.remove(pcup)
        idx_dest = cups.index(dest)
        for pcup in reversed(picked_up):
            cups.insert(idx_dest+1,  pcup)
        idx = (cups.index(current_cup) + 1) % len(cups)
            
    return cups

final_cups = do_moves(test_input, 10, verbose=True)

[3, 8, 9, 1, 2, 5, 4, 6, 7] 0 2 [8, 9, 1]
[3, 2, 8, 9, 1, 5, 4, 6, 7] 1 7 [8, 9, 1]
[3, 2, 5, 4, 6, 7, 8, 9, 1] 2 3 [4, 6, 7]
[3, 4, 6, 7, 2, 5, 8, 9, 1] 6 7 [9, 1, 3]
[4, 6, 7, 9, 1, 3, 2, 5, 8] 0 3 [6, 7, 9]
[4, 1, 3, 6, 7, 9, 2, 5, 8] 1 9 [3, 6, 7]
[4, 1, 9, 3, 6, 7, 2, 5, 8] 2 8 [3, 6, 7]
[4, 1, 9, 2, 5, 8, 3, 6, 7] 3 1 [5, 8, 3]
[4, 1, 5, 8, 3, 9, 2, 6, 7] 7 5 [7, 4, 1]
[5, 7, 4, 1, 8, 3, 9, 2, 6] 0 3 [7, 4, 1]


In [64]:
idx1 = final_cups.index(1)
assert ''.join([str(final_cups[(idx1 + i) % len(final_cups)]) for i in range(1, len(final_cups))]) == '92658374'

In [65]:
final_cups = do_moves(test_input, 100)

idx1 = final_cups.index(1)
assert ''.join([str(final_cups[(idx1 + i) % len(final_cups)]) for i in range(1, len(final_cups))]) == '67384529'

In [66]:
final_cups = do_moves(input23, 100)

idx1 = final_cups.index(1)
''.join([str(final_cups[(idx1 + i) % len(final_cups)]) for i in range(1, len(final_cups))])

'96342875'

## Part 2 

In [79]:
extended_input23 = input23.copy()
extended_input23.extend(range(len(input23)+1, 1000001))
assert len(np.unique(extended_input23)) == 1e6

In [83]:
%time final_cups = do_moves(extended_input23, 100)

CPU times: user 1.09 s, sys: 0 ns, total: 1.09 s
Wall time: 1.09 s


In [87]:
minutes_estimate = 1e6/100/60
minutes_estimate

166.66666666666666

Eh, I can let it run for a few hours.

In [110]:
from tqdm.notebook import tqdm

extended_input23 = input23.copy()
extended_input23.extend(range(len(input23)+1, 1000001))

final_cups = do_moves(extended_input23, 1000000, progress=tqdm)

idx1 = final_cups.index(1)
cup2, cup3 = final_cups[idx1+1:idx1+3]
cup2 * cup3

70

Oops, misread... it's 10 million. Need to profile what's slow...

In [382]:
%load_ext line_profiler

lprof = %lprun -r -f do_moves do_moves(extended_input23, 100)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


~80% of the time is in cups.index(dest) and ~12% is in cups.remove ... suggests numpy might do better

In [467]:
def do_moves_faster(icups, moves=100, verbose=False, progress=None):
    cups = np.array(icups, copy=True)
    
    mincups = np.min(cups)
    maxcups = np.max(cups)
    
    if progress is None:
        progress = lambda x:x
    
    current_idx = 0
    for i in progress(range(moves)):
        try:
            if current_idx >= (len(cups)-3):
                cups = np.roll(cups, -3)
                current_idx -= 3
            current_cup = cups[current_idx]
            picked_up = cups[current_idx+1:current_idx+4]
            head = cups[:current_idx+1]
            tail = cups[current_idx+4:]

            dest = current_cup - 1
            if dest < mincups:
                dest = maxcups
            while dest in picked_up:
                dest -= 1
                if dest < mincups:
                    dest = maxcups
            if verbose:
                print(cups, current_idx, dest, picked_up)

            spliced = np.concatenate((head, tail))

            idx_dest = np.where(dest==spliced)[0][0] +1
        except:
            fn = 'checkfile_day23'
            print(f'Writing out check point file {fn}...')
            np.savez(fn, cups=cups, i=np.array([i]), icups=np.array(icups))
            raise
        cups = np.concatenate((spliced[:idx_dest], picked_up, spliced[idx_dest:]))
        current_idx = (np.where(cups==current_cup)[0][0] + 1) % len(cups)

    return cups

%time do_moves(extended_input23, 100, verbose=False)
%time do_moves_faster(extended_input23, 100, verbose=False)
do_moves(test_input, 10, verbose=True)
do_moves_faster(test_input, 10, verbose=True)

CPU times: user 1.08 s, sys: 0 ns, total: 1.08 s
Wall time: 1.07 s
CPU times: user 375 ms, sys: 0 ns, total: 375 ms
Wall time: 374 ms
[3, 8, 9, 1, 2, 5, 4, 6, 7] 0 2 [8, 9, 1]
[3, 2, 8, 9, 1, 5, 4, 6, 7] 1 7 [8, 9, 1]
[3, 2, 5, 4, 6, 7, 8, 9, 1] 2 3 [4, 6, 7]
[3, 4, 6, 7, 2, 5, 8, 9, 1] 6 7 [9, 1, 3]
[4, 6, 7, 9, 1, 3, 2, 5, 8] 0 3 [6, 7, 9]
[4, 1, 3, 6, 7, 9, 2, 5, 8] 1 9 [3, 6, 7]
[4, 1, 9, 3, 6, 7, 2, 5, 8] 2 8 [3, 6, 7]
[4, 1, 9, 2, 5, 8, 3, 6, 7] 3 1 [5, 8, 3]
[4, 1, 5, 8, 3, 9, 2, 6, 7] 7 5 [7, 4, 1]
[5, 7, 4, 1, 8, 3, 9, 2, 6] 0 3 [7, 4, 1]
[3 8 9 1 2 5 4 6 7] 0 2 [8 9 1]
[3 2 8 9 1 5 4 6 7] 1 7 [8 9 1]
[3 2 5 4 6 7 8 9 1] 2 3 [4 6 7]
[7 2 5 8 9 1 3 4 6] 3 7 [9 1 3]
[3 2 5 8 4 6 7 9 1] 4 3 [6 7 9]
[9 2 5 8 4 1 3 6 7] 5 9 [3 6 7]
[9 3 6 7 2 5 8 4 1] 0 8 [3 6 7]
[9 2 5 8 3 6 7 4 1] 1 1 [5 8 3]
[9 2 6 7 4 1 5 8 3] 2 5 [7 4 1]
[9 2 6 5 7 4 1 8 3] 3 3 [7 4 1]


array([9, 2, 6, 5, 8, 3, 7, 4, 1])

Still will take a long time, but "overnight" is fine.

In [468]:
from tqdm.notebook import tqdm

extended_input23 = input23.copy()
extended_input23.extend(range(len(input23)+1, 1000001))

final_cups = do_moves_faster(extended_input23, 10000000, progress=tqdm)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [472]:
idx1 = np.where(final_cups == 1)[0][0]
cup2, cup3 = final_cups[idx1+1:idx1+3]
cup2 * cup3

563362809504

In [476]:
np.savez('end_state_day23', cups=final_cups, i=np.array([10000000]), icups=np.array(extended_input23))